In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [7]:
df=pd.read_csv('dataset_part_2.csv')

In [8]:
df.head(1)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,1


In [9]:
import folium

# Create a base map centered around the average coordinates
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
launch_map = folium.Map(location=map_center, zoom_start=5)

# Get unique launch sites with their coordinates
unique_sites = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()

# Add markers for each launch site
for idx, row in unique_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['LaunchSite'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(launch_map)

# Display the map
launch_map


In [20]:
spacex_df=pd.read_csv('spacex_launch_geo.csv')

In [21]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [22]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [24]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [30]:
import folium
from folium.features import DivIcon

# Create base map centered around average location
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
site_map = folium.Map(location=map_center, zoom_start=5)

# Iterate over unique launch sites and add circles + markers
for idx, row in df.iterrows():
    coord = (row['Latitude'], row['Longitude'])
    site_name = row['LaunchSite']
    
    # Add a circle with radius 1000 meters, yellow color
    folium.Circle(
        location=coord,
        radius=1000,
        color='#ffff00',  # yellow color
        fill=True,
        fill_color='#ffff00',
        fill_opacity=0.4
    ).add_child(
        folium.Popup(site_name)
    ).add_to(site_map)
    
    # Add a labeled marker with custom DivIcon (orange label)
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

site_map # Display map in Jupyter notebook


In [31]:
# Assuming df has 'Latitude', 'Longitude', 'LaunchSite', 'Class' columns

for idx, row in df.iterrows():
    coord = (row['Latitude'], row['Longitude'])
    site_name = row['LaunchSite']
    outcome = row['Class']  # 1 for success, 0 for failure
    
    # Color based on success or failure
    color = 'green' if outcome == 1 else 'red'
    
    folium.CircleMarker(
        location=coord,
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{site_name} - {'Success' if outcome == 1 else 'Failure'}"
    ).add_to(site_map)

site_map  # Display the updated map


In [32]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [34]:
marker_cluster = MarkerCluster()


In [37]:
# Using lambda function
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')



In [39]:
def get_marker_color(cls):
    return 'green' if cls == 1 else 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)


In [54]:
# Create a map centered at average coordinates (adjust as needed)
site_map = folium.Map(location=[28.5, -80.5], zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Iterate over each row in the dataframe
for index, record in spacex_df.iterrows():
    # Create a marker with a popup showing Launch Site and Outcome
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket')
    )
    # Add marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

In [55]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [56]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [57]:
# Launch site coordinates
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Closest coastline coordinates
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to coastline: {distance_coastline:.2f} km")

Distance to coastline: 0.58 km


In [58]:
# Coordinates
launch_site_coord = [28.562302, -80.577356]
coastline_coord = [28.56367, -80.57163]

# Calculate distance
distance = calculate_distance(launch_site_coord[0], launch_site_coord[1], coastline_coord[0], coastline_coord[1])

# Create map centered at launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=14)

# Add marker for launch site
folium.Marker(launch_site_coord, popup='Launch Site', icon=folium.Icon(color='blue')).add_to(site_map)

# Add marker for coastline
folium.Marker(coastline_coord, popup='Coastline', icon=folium.Icon(color='green')).add_to(site_map)

# Add distance label at midpoint between the two
midpoint = [(launch_site_coord[0] + coastline_coord[0]) / 2,
            (launch_site_coord[1] + coastline_coord[1]) / 2]

# Add distance marker using DivIcon
distance_marker = folium.Marker(
    midpoint,
    icon=DivIcon(
        icon_size=(100, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),
    )
)

# Add everything to map
distance_marker.add_to(site_map)

# Show map
site_map


In [59]:

# Define coordinates
launch_site_coord = [28.562302, -80.577356]
coastline_coord = [28.56367, -80.57163]

# Create a map centered at the launch site
site_map = folium.Map(location=launch_site_coord, zoom_start=14)

# Add markers
folium.Marker(launch_site_coord, popup='Launch Site', icon=folium.Icon(color='blue')).add_to(site_map)
folium.Marker(coastline_coord, popup='Coastline', icon=folium.Icon(color='green')).add_to(site_map)

# Create polyline
coordinates = [launch_site_coord, coastline_coord]
lines = folium.PolyLine(locations=coordinates, weight=2, color='red')
site_map.add_child(lines)

# Display the map
site_map


In [61]:
# Helper function to calculate distance (in KM)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Launch site coordinates (e.g., CCAFS LC-40)
launch_coord = [28.562302, -80.577356]

# Example: Closest city (e.g., Cocoa Beach)
city_coord = [28.320006, -80.607552]  # Replace with actual coordinates of the closest city

# Calculate distance
distance_km = calculate_distance(launch_coord[0], launch_coord[1], city_coord[0], city_coord[1])

# Create the map centered at the launch site
site_map = folium.Map(location=launch_coord, zoom_start=9)

# Add launch site marker
folium.Marker(launch_coord, popup='Launch Site', icon=folium.Icon(color='blue')).add_to(site_map)

# Add closest feature marker (e.g., city)
folium.Marker(city_coord, popup='Closest City',
              icon=folium.Icon(color='green')).add_to(site_map)

# Add a line between launch site and the closest city
folium.PolyLine(locations=[launch_coord, city_coord], color='red', weight=2).add_to(site_map)

# Add a distance label near the midpoint of the line
midpoint = [(launch_coord[0] + city_coord[0]) / 2, (launch_coord[1] + city_coord[1]) / 2]
folium.Marker(
    location=midpoint,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 14px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# Show the map
site_map
